<h1 style='background-color:#2E8B57; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 15px 50px;' > EDA+Google Smartphone Decimeter  </h1>

### What is the EDA

Exploratory Data Analysis refers to the critical process of performing initial investigations on data so as to discover patterns,to spot anomalies,to test hypothesis and to check assumptions with the help of summary statistics and graphical representations.

<img src="https://cdn.educba.com/academy/wp-content/uploads/2020/01/exploratory-data-analysis.jpg" width="800px">







<h1 style='background-color:#2E8B57; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 15px 50px;' > Google Smartphone Decimeter Challenge  </h1>

### Improve high precision GNSS positioning and navigation accuracy on smartphones


<img src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F603584%2F88764e97cd9c2df83195454039b9e544%2Fsmartphone-dec.png?generation=1620664283744618&alt=media" width="500px">




Global Navigation Satellite System (GNSS) provides raw signals, which the GPS chipset uses to compute a position. Current mobile phones only offer 3-5 meters of positioning accuracy. While useful in many cases, it can create a “jumpy” experience. For many use cases the results are not fine nor stable enough to be reliable.

This competition, hosted by the Android GPS team, is being presented at the ION GNSS+ 2021 Conference. They seek to advance research in smartphone GNSS positioning accuracy and help people better navigate the world around them.


### Data Description

[train]/[drive_id]/[phone_name]/ground_truth.csv - Only provided for the training set. Reference locations at expected timestamps.


[train/test]/[drive_id]/[phone_name]/supplemental/[phone_name][.20o/.21o/.nmea] - Equivalent data to the gnss logs in other formats used by the GPS community.


baseline_locations_[train/test].csv - Estimated coordinates generated using a simple approach.

#### Dataset Link 

##### [Here](https://www.kaggle.com/c/google-smartphone-decimeter-challenge/data)

In [ ]:
!pip install dataprep by

In [ ]:
pip install autoviz

In [ ]:
!pip install xlrd

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from pathlib import Path
import numpy as np
from tqdm.notebook import tqdm
from dataprep.eda import *
from dataprep.eda import plot
from dataprep.eda import plot_diff
from dataprep.eda import plot_correlation
from dataprep.eda import plot_missing
import plotly.express as px
import plotly.figure_factory as ff
import warnings
warnings.filterwarnings("ignore")
PH = "../input/google-smartphone-decimeter-challenge/"

In [ ]:
train = pd.read_csv(PH+'baseline_locations_train.csv')
test = pd.read_csv(PH+'baseline_locations_test.csv')
submission = pd.read_csv(PH+'sample_submission.csv')
submission = submission.assign(latDeg = test.latDeg.round(6),lngDeg = test.lngDeg.round(6))
submission.to_csv('submission.csv', index=False)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
submission.info()

In [ ]:
train

In [ ]:
plot_diff([train, test])

In [ ]:
plot_correlation(train)

In [ ]:
plot(train, "phoneName")

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(test)

In [ ]:
create_report(train)

In [ ]:
create_report(test)

In [ ]:
train

In [ ]:
datapath = Path("../input/google-smartphone-decimeter-challenge")

truths = (datapath / 'train').rglob('ground_truth.csv')

In [ ]:
df_list = []
cols = ['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg','lngDeg']

for t in tqdm(truths, total=73):
    df_phone = pd.read_csv(t, usecols=cols)  
    df_list.append(df_phone)
df_truth = pd.concat(df_list, ignore_index=True)

df_basepreds = pd.read_csv('../input/google-smartphone-decimeter-challenge/baseline_locations_train.csv', usecols=cols)
df_all = df_truth.merge(df_basepreds, how='inner', on=cols[:3], suffixes=('_truth', '_basepred'))
df_all[:100]

In [ ]:
# https://plotly.com/python/mapbox-layers/
import plotly.express as px
def MAP(df_all, center):
    fig = px.scatter_mapbox(df_all,lat="latDeg", lon="lngDeg", color="phoneName",labels="phoneName",zoom=9,center=center,height=500, width=600)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [ ]:
Pixel4 = pd.read_csv(PH + "train/2020-05-14-US-MTV-1/Pixel4/ground_truth.csv")

center = {"lat":37.428281, "lon":-122.072541}
MAP(Pixel4, center)

In [ ]:
Pixel4XLModded = pd.read_csv(PH + "train/2020-05-14-US-MTV-1/Pixel4XLModded/ground_truth.csv")

center = {"lat":37.6901944483, "lon":-122.3923280950}
MAP(Pixel4XLModded, center)

In [ ]:
Mi8 = pd.read_csv(PH + "train/2020-07-17-US-MTV-1/Mi8/ground_truth.csv")

center = {"lat":37.4282808217, "lon":-122.0725408417}
MAP(Mi8, center)

In [ ]:
Pixel5 = pd.read_csv(PH + "train/2021-01-04-US-RWC-2/Pixel5/ground_truth.csv")

center = {"lat":37.4443845150, "lon":-122.2326749300}
MAP(Pixel5, center)

In [ ]:
SamsungS20Ultra = pd.read_csv(PH + "train/2021-03-10-US-SVL-1/SamsungS20Ultra/ground_truth.csv")

center = {"lat":37.3713148617, "lon":-122.04798264007}
MAP(SamsungS20Ultra, center)

In [ ]:
All = pd.concat([Pixel4, Pixel4XLModded, Mi8,Pixel5,SamsungS20Ultra])
MAP(All, center)

In [ ]:
df_all.info()

In [ ]:
df_all.isna()

In [ ]:
df_all.isnull().sum()

In [ ]:
# Chack if we have miss data
plot_missing(df_all)

In [ ]:
df_all

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
AV = AutoViz_Class()
target='phoneName'
df = AV.AutoViz(filename="",sep=',', depVar=target, dfte=df_all, header=0, verbose=2, 
                 lowess=False, chart_format='svg', max_rows_analyzed=150000, max_cols_analyzed=30)

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(df_all)

In [ ]:
plot(df_all)

In [ ]:
plot(df_all, 'lngDeg_basepred')

In [ ]:
plot(df_all, 'latDeg_basepred')

In [ ]:
plot(df_all, 'lngDeg_truth')

In [ ]:
plot(df_all, 'millisSinceGpsEpoch')

In [ ]:
plot(df_all, 'phoneName')

In [ ]:

plot(df_all, 'collectionName')

In [ ]:
create_report(df_all)

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(df_all.corr(), annot=True)

In [ ]:
df_all[:10].T

In [ ]:
phoneNamene = {'Pixel4':1, 
        'Pixel4XL':2, 
        'SamsungS20Ultra':3, 
        'Mi8':4, 
        'Pixel5':5, 
        'Pixel4XLModded':6,
        'Pixel4Modded':7}

# apply using map
df_all['phoneName'] = df_all['phoneName'].map(phoneNamene)

In [ ]:
df_all[:10]

In [ ]:
df_all.info()

In [ ]:
plot_correlation(df_all)

## Good Luck All